In [1]:
import os
import cv2
import imutils
import numpy as np
from imutils import paths
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Dense
from SpatialPyramidPooling import SpatialPyramidPooling
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import train_test_split, KFold
from sklearn.cross_validation import cross_val_score,cross_val_predict
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import confusion_matrix,accuracy_score 
from sklearn.metrics import classification_report

Using TensorFlow backend.
C:\Users\Gloria Hutauruk\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
imagePaths = list(paths.list_images("data_gambar"))

images = []
labels = []
for (i, imagePath) in enumerate(imagePaths):
    image = cv2.imread(imagePath)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    label = imagePath.split(os.path.sep)[-1].split(".")[0]
    images.append(hsv)
    labels.append(label)
    
    if i > 0 and ((i + 1)% 200 == 0 or i ==len(imagePaths)-1):
        print("[INFO] processed {}/{}".format(i+1, len(imagePaths)))

images = np.array(images)
labels = np.array(labels)

num_channels = len(images[0].shape)
num_classes = len(set(labels))

model = Sequential()
model.add(SpatialPyramidPooling([1, 2, 4, 8, 16, 32], input_shape=(None, None, num_channels)))
model.compile(loss='mse', optimizer='sgd')

spp_visual_feature = []

for index, values in np.ndenumerate(images):
    im = np.expand_dims(values, axis=0)
    x = model.predict(im)
    print("[INFO] Features extracted from Image ", *index)
    spp_visual_feature.append(x[0])

spp_hsv_visual_feature = np.array(spp_visual_feature)



[INFO] processed 200/700
[INFO] processed 400/700
[INFO] processed 600/700
[INFO] processed 700/700
[INFO] Features extracted from Image  0
[INFO] Features extracted from Image  1
[INFO] Features extracted from Image  2
[INFO] Features extracted from Image  3
[INFO] Features extracted from Image  4
[INFO] Features extracted from Image  5
[INFO] Features extracted from Image  6
[INFO] Features extracted from Image  7
[INFO] Features extracted from Image  8
[INFO] Features extracted from Image  9
[INFO] Features extracted from Image  10
[INFO] Features extracted from Image  11
[INFO] Features extracted from Image  12
[INFO] Features extracted from Image  13
[INFO] Features extracted from Image  14
[INFO] Features extracted from Image  15
[INFO] Features extracted from Image  16
[INFO] Features extracted from Image  17
[INFO] Features extracted from Image  18
[INFO] Features extracted from Image  19
[INFO] Features extracted from Image  20
[INFO] Features extracted from Image  21
[INFO] F

In [3]:
pca = PCA(n_components = 1)
spp_hsv_pca_feature = pca.fit_transform(spp_hsv_visual_feature)



In [5]:
mattrix_image = pd.DataFrame(spp_hsv_pca_feature,labels)
mattrix_image

,0
bestseller,-2489.885986
bestseller,725.606873
bestseller,705.940063
bestseller,-2018.141602
bestseller,-1706.871094
bestseller,1.273865
bestseller,1417.473877
bestseller,211.304077
bestseller,2112.116943
bestseller,3178.535156


In [4]:
import csv
sentimen_score = []
with open('data_fitur_sentimen.csv') as csvFile:
    spamreader = csv.reader(csvFile, delimiter = ',')
    for row in spamreader:
        sentimen_score.append(float(row[0]))

In [5]:
combine_feature = [None]*len(spp_hsv_pca_feature)


In [6]:
for i in range(len(spp_hsv_pca_feature)):
    combine_feature[i] = np.hstack((spp_hsv_pca_feature[i],sentimen_score[i]))


In [7]:
fitur_combine = np.array(combine_feature)

In [8]:
category_encoder = LabelEncoder()
y = category_encoder.fit_transform(labels)

In [9]:
C = 1.0  # SVM regularization parameter
model_combine = SVC(kernel='linear', C=C)
model_kombinasi_fitur = model_combine.fit(fitur_combine,labels)

In [10]:
p = cross_val_score(model_kombinasi_fitur, fitur_combine, y, cv=10, scoring= 'precision')
precision_combine_feature = (sum(p)/len(p))
r = cross_val_score(model_kombinasi_fitur, fitur_combine, y, cv=10, scoring='recall')
recall_combine_feature = (sum(r)/len(r)) 
f = cross_val_score(model_kombinasi_fitur, fitur_combine, y, cv=10, scoring='f1')
f_measure_combine_feature = (sum(f)/len(f)) 


In [11]:
precision_combine_feature,recall_combine_feature,f_measure_combine_feature

(0.49361946734390871, 0.41999999999999993, 0.44813968577665009)